# Парсинг с помощью BeautifulSoup

Beautiful Soup - это библиотека для Python, которая позволяет парсить (анализировать) HTML и XML документы. Она предоставляет удобный способ искать, навигировать, и модифицировать дерево DOM (Document Object Model), представляющее HTML/XML документ.

# Задание

Вам необходимо собрать датасет, спарсив данные из этого сайта:

https://books.toscrape.com/

Всего на сайте 1000 книг. То есть длина датасета должна равняться количеству книг.
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |

**Примечание по столбцам:**
- `id` - заполняется разработчиком датасета. Первая спарсенная книга имеет `id` = `0`.
- `url` - должна содержать полную ссылку. Не только конец ссылки, указанный на сайте. То есть по данному url можно перейти одним кликом.

## Импорт библиотек

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Cоздание датасета и парсинг данных

In [2]:
#Создаем пустой датафрейм
df = pd.DataFrame({
    'id':[],
    'book_name':[],
    'price':[],
    'stock':[],
    'url':[],
})

id_ = 0

#функция наполнения датафрейма
def books_df (soup, data = df):
    articles = soup.find_all('article', attrs = {'class':'product_pod'})
    global id_
    for i in articles:
        quote = {}
        quote['id'] = id_
        quote['book_name'] = i.find('h3').text 
        # удалил из цены без f и изменил тип на числовой, чтобы можно было анализировать
        quote['price'] = float(i.find('p', attrs = {'class':'price_color'}).text[1:])  
        
        if i.find('p', attrs = {'class':'instock availability'}).text.strip() == 'In stock':
            quote['stock'] = 1
        else: 
            quote['stock'] = 0  
            
        quote['url'] = 'https://books.toscrape.com/catalogue/' + i.find('h3').find('a').get('href')
        
        df.loc[len(df)] = quote
        id_ += 1

#Парсинг сайта для наполнения датафрейма        

url = 'http://books.toscrape.com/'

for i in range(1, 51): #На сайте 50 страниц (1000 книг по 20 на странице)
    r = requests.get(url + 'catalogue/page-'+ str(i) + '.html', timeout = 10)
    soup = BeautifulSoup(r.content, 'html.parser')    
    books_df(soup, df)  


## Итоговый датасет

In [3]:
df.shape

(1000, 5)

In [4]:
display(df.head(), df.tail())

,id,book_name,price,stock,url
0,0,A Light in the ...,51.77,1,https://books.toscrape.com/catalogue/a-light-i...
1,1,Tipping the Velvet,53.74,1,https://books.toscrape.com/catalogue/tipping-t...
2,2,Soumission,50.10,1,https://books.toscrape.com/catalogue/soumissio...
3,3,Sharp Objects,47.82,1,https://books.toscrape.com/catalogue/sharp-obj...
4,4,Sapiens: A Brief History ...,54.23,1,https://books.toscrape.com/catalogue/sapiens-a...


,id,book_name,price,stock,url
995,995,Alice in Wonderland (Alice's ...,55.53,1,https://books.toscrape.com/catalogue/alice-in-...
996,996,"Ajin: Demi-Human, Volume 1 ...",57.06,1,https://books.toscrape.com/catalogue/ajin-demi...
997,997,A Spy's Devotion (The ...,16.97,1,https://books.toscrape.com/catalogue/a-spys-de...
998,998,1st to Die (Women's ...,53.98,1,https://books.toscrape.com/catalogue/1st-to-di...
999,999,"1,000 Places to See ...",26.08,1,https://books.toscrape.com/catalogue/1000-plac...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1000 non-null   int64  
 1   book_name  1000 non-null   object 
 2   price      1000 non-null   float64
 3   stock      1000 non-null   int64  
 4   url        1000 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 46.9+ KB


# ЗАДАНИЕ ПРО

Так, мы спарсили данные о книгах. Но данные какие-то неполные. Часть названия стирается из-за отображения и нет ни полного названия книги, ни описания этой книги, ни жанра.

Вам необходимо дополнить датасет, спарсив дополнительные данные из того же сайта:

https://books.toscrape.com/
 
Итоговая таблица должна содержать следующие столбцы:

| Название столбца | Описание | 
|--|--|
|id| Идентификатор книги |
|book_name| Название книги - только полное название|
|genre| жанр книги |
|desc| описание |
|price| Цена в £ |
|stock| Наличие книги. 1 или 0|
|url| Ссылка на книгу |
| num_of_rev | количество отзывов|

## Парсинг данных и обогащение датасета

In [6]:
#Вставляем в середину два столбца
df.insert(2, 'genre', '', allow_duplicates = False)
df.insert(3, 'desc', '', allow_duplicates = False)

In [10]:
#Парсим страницы книг и добавляем данные из них в датасет
n = 0
for i in df['url']:
    r = requests.get(i, verify=True, timeout=10)
    soup = BeautifulSoup(r.content, 'html.parser')
    content = soup.find_all('div', attrs = {'class':'container-fluid page'})
    df.at[n, 'genre'] = (content[0].find_all('a'))[2].text
    df.at[n,'book_name'] = content[0].find('h1').text
    df.at[n,'desc'] = (content[0].find_all('p'))[3].text
    df.at[n,'num_of_rev'] = int((content[0].find_all('td'))[6].text) 
    n += 1


## Итоговый датасет PRO

In [11]:
df.shape

(1000, 8)

In [12]:
display(
    df.head(),
    df.tail()
)

,id,book_name,genre,desc,price,stock,url,num_of_rev
0,0,A Light in the Attic,Poetry,It's hard to imagine a world without A Light i...,51.77,1,https://books.toscrape.com/catalogue/a-light-i...,0.0
1,1,Tipping the Velvet,Historical Fiction,"""Erotic and absorbing...Written with starling ...",53.74,1,https://books.toscrape.com/catalogue/tipping-t...,0.0
2,2,Soumission,Fiction,"Dans une France assez proche de la nôtre, un h...",50.10,1,https://books.toscrape.com/catalogue/soumissio...,0.0
3,3,Sharp Objects,Mystery,"WICKED above her hipbone, GIRL across her hear...",47.82,1,https://books.toscrape.com/catalogue/sharp-obj...,0.0
4,4,Sapiens: A Brief History of Humankind,History,From a renowned historian comes a groundbreaki...,54.23,1,https://books.toscrape.com/catalogue/sapiens-a...,0.0


,id,book_name,genre,desc,price,stock,url,num_of_rev
995,995,Alice in Wonderland (Alice's Adventures in Won...,Classics,\n\n\n\n\n\n,55.53,1,https://books.toscrape.com/catalogue/alice-in-...,0.0
996,996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Sequential Art,High school student Kei Nagai is struck dead i...,57.06,1,https://books.toscrape.com/catalogue/ajin-demi...,0.0
997,997,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,"In England’s Regency era, manners and elegance...",16.97,1,https://books.toscrape.com/catalogue/a-spys-de...,0.0
998,998,1st to Die (Women's Murder Club #1),Mystery,"James Patterson, bestselling author of the Ale...",53.98,1,https://books.toscrape.com/catalogue/1st-to-di...,0.0
999,999,"1,000 Places to See Before You Die",Travel,"Around the World, continent by continent, here...",26.08,1,https://books.toscrape.com/catalogue/1000-plac...,0.0
